# NLP - Traitement Automatique du Langage Naturel

Notre librarie de code préférée pour le traitement du langage naturel est [SpaCy](../tools/spacy.ipynb), et nous lui avons d'ailleurs dédié un bloc-note Jupyter qui démontre les bases.

Les [transformers](https://arxiv.org/abs/1706.03762) ont changé les paradigmes du traitement du langage naturel. Il existe de nombreux modèles anglais et multilingues, mais peu de modèles français, l'inconvénient des modèles multilingues étant principalement leur taille, et accessoirement les faiblesses de la part du corpus d'entrainement dédiée au français. Nous avons donc souhaité rendre ici hommage aux rares modèles dédiés au français, que sont [Camembert](https://huggingface.co/almanach), [Croissant](https://huggingface.co/croissantllm), [Flaubert](https://huggingface.co/flaubert), [Vigogne](https://huggingface.co/bofenghuang), et bientôt Lucie.

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/algorithms/nlp-basics.ipynb)

In [3]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U torch
# !pip3 install -q -U transformers

# À installer dans tous les cas pour Google Colab et Github
if 'google.colab' in sys.modules or 'CI' in os.environ:
    !pip3 install -q -U sacremoses # requis par Flaubert

In [ ]:
# Camembert
camembert_name = "camembert/camembert-base"
# Croissant
croissant_name = "croissantllm/CroissantLLMBase"
# Flaubert
flaubert_name = "flaubert/flaubert_base_cased"

## Tokenizers

In [9]:
from transformers import AutoTokenizer, CamembertTokenizer, FlaubertTokenizer

# Camembert
camembert_tokenizer = CamembertTokenizer.from_pretrained(camembert_name)
# Croissant
# croissant_tokenizer = AutoTokenizer.from_pretrained(croissant_name)
# Flaubert
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(flaubert_name, do_lowercase=False)


In [10]:
print(camembert_tokenizer)

CamembertTokenizer(name_or_path='camembert/camembert-base', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<unk>NOTUSED']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>NOTUSED", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>NOTUSED", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<unk>NOTUSED", rstrip=False, lstrip=False, single_word=False, no

In [ ]:
#print(croissant_tokenizer)

In [11]:
print(flaubert_tokenizer)

FlaubertTokenizer(name_or_path='flaubert/flaubert_base_cased', vocab_size=68729, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '</s>', 'mask_token': '<special1>', 'additional_special_tokens': ['<special0>', '<special1>', '<special2>', '<special3>', '<special4>', '<special5>', '<special6>', '<special7>', '<special8>', '<special9>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<special0>", rstrip=Fals

In [22]:
def check_vocab(name='<s>', tokenizer=camembert_tokenizer):
    vocab = tokenizer.get_vocab()
    try:
        id = vocab[name]
        print(f"{name} est dans le vocabulaire de {tokenizer.name_or_path}: {id}")
    except KeyError:
        print(f"{name} n'est pas dans le vocabulaire de {tokenizer.name_or_path}")

# Camembert
check_vocab()
check_vocab('chat')
check_vocab('chien')
check_vocab('balle')
# print()
# Croissant
# check_vocab(tokenizer=croissant_tokenizer)
# check_vocab('chat', croissant_tokenizer)
# check_vocab('chien', croissant_tokenizer)
# check_vocab('balle', croissant_tokenizer)
print()
# Flaubert
check_vocab(tokenizer=flaubert_tokenizer)
check_vocab('chat', flaubert_tokenizer)
check_vocab('chien', flaubert_tokenizer)
check_vocab('balle', flaubert_tokenizer)

<s> est dans le vocabulaire de camembert/camembert-base: 5
chat est dans le vocabulaire de camembert/camembert-base: 8734
chien n'est pas dans le vocabulaire de camembert/camembert-base
balle n'est pas dans le vocabulaire de camembert/camembert-base

<s> est dans le vocabulaire de flaubert/flaubert_base_cased: 0
chat est dans le vocabulaire de flaubert/flaubert_base_cased: 8830
chien n'est pas dans le vocabulaire de flaubert/flaubert_base_cased
balle n'est pas dans le vocabulaire de flaubert/flaubert_base_cased


## Embeddings

In [ ]:
import torch
from transformers import AutoModelForCausalLM, CamembertModel, FlaubertModel

# Camembert
camembert_model = CamembertModel.from_pretrained(camembert_name)
# Croissant
# croissant_model = AutoModelForCausalLM.from_pretrained(croissant_name, torch_dtype=torch.float16, device_map="auto")
# Flaubert
flaubert_model = FlaubertModel.from_pretrained(flaubert_name)

## Modèles

In [ ]:
import torch
from transformers import AutoModelForCausalLM, CamembertModel, FlaubertModel

# Camembert
camembert_model = CamembertModel.from_pretrained(camembert_name)
# Croissant
# croissant_model = AutoModelForCausalLM.from_pretrained(croissant_name, torch_dtype=torch.float16, device_map="auto")
# Flaubert
flaubert_model = FlaubertModel.from_pretrained(flaubert_name)

In [ ]:
from transformers import pipeline

nlp_fill_camembert = pipeline('fill-mask', model=camembert_name, top_k=10)
#nlp_fill_croissant = pipeline('fill-mask', model=croissant_name, top_k=10)
nlp_fill_flaubert = pipeline('fill-mask', model=flaubert_name, top_k=10)

In [ ]:
# https://medium.com/@xiaoouwang/playing-with-camembert-and-flaubert-8c5d40e502a7

In [ ]:
#inputs = tokenizer("I am so tired I could sleep right now. -> Je suis si fatigué que je pourrais m'endormir maintenant.\nHe is heading to the market. -> Il va au marché.\nWe are running on the beach. ->", return_tensors="pt").to(model.device)
#tokens = model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60, temperature=0.3)
#print(tokenizer.decode(tokens[0]))

# remove bos token
#inputs = tokenizer("Capitales: France -> Paris, Italie -> Rome, Allemagne -> Berlin, Espagne ->", return_tensors="pt", add_special_tokens=True).to(model.device)
#tokens = model.generate(**inputs, max_length=100, do_sample=True, top_p=0.95, top_k=60)
#print(tokenizer.decode(tokens[0]))

In [2]:
# Pour supprimer les modèles du cache
# !pip3 install -q -U "huggingface_hub[cli]"
# !huggingface-cli scan-cache
# !huggingface-cli delete-cache

REPO ID                      REPO TYPE SIZE ON DISK NB FILES LAST_ACCESSED LAST_MODIFIED REFS LOCAL PATH                                                                    
---------------------------- --------- ------------ -------- ------------- ------------- ---- ----------------------------------------------------------------------------- 
camembert/camembert-base     model           446.0M        3 22 hours ago  22 hours ago  main /Users/jlchereau/.cache/huggingface/hub/models--camembert--camembert-base     
flaubert/flaubert_base_cased model           555.7M        5 22 hours ago  22 hours ago  main /Users/jlchereau/.cache/huggingface/hub/models--flaubert--flaubert_base_cased 

Done in 0.0s. Scanned 2 repo(s) for a total of 1.0G.
Got 2 warning(s) while scanning. Use -vvv to print details.
